In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import RegexpTokenizer
from sklearn.linear_model import SGDClassifier
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SpatialDropout1D, LSTM, GlobalMaxPool1D
redditData = pd.read_csv("/content/reddit_train.csv") #, sep="\n", header=None) 

X = redditData.iloc[:,1]
Y = redditData.iloc[:,-1]
Y = pd.get_dummies(Y).values

X = [str (item) for item in X]
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.30, random_state = 42)

print(X_train)
print(y_train)

MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 20
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)

model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(GlobalMaxPool1D())
model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train,epochs=5,verbose=False,batch_size=10)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1000, 100)         500000    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_6 (Dense)              (None, 20)                1020      
Total params: 506,070
Trainable params: 506,070
Non-trainable params: 0
_________________________________________________________________


In [0]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9660
Testing Accuracy:  0.9615


In [0]:
redditDataTest = pd.read_csv("/content/reddit_test.csv") #, sep="\n", header=None) 

commentsTestTF = redditDataTest.iloc[:,1]
commentsTestTF = [str (item) for item in commentsTestTF]
commentsTestTF = tokenizer.texts_to_sequences(commentsTestTF)
commentsTestTF = pad_sequences(commentsTestTF, maxlen=MAX_SEQUENCE_LENGTH)

pred = model.predict(np.array(commentsTestTF))

In [0]:
s = pd.Series(subredditsTrain).astype('category')

df = pd.get_dummies(s)
convert = df.columns 

In [0]:
results = []
for x in pred:
  results.append(convert[np.argmax(x)])

In [0]:
pred = pd.DataFrame(results, columns=['Category']).to_csv("testResultsNN.csv")

In [88]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import numbers
import decimal
import scipy.stats as ss
import matplotlib.pyplot as plt
from statistics import stdev
from statistics import mean
import time
import math
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
import string

redditDataTrain = pd.read_csv("/content/reddit_train.csv") #, sep="\n", header=None) 
commentsTrain = redditDataTrain.iloc[:,1]
subredditsTrain = redditDataTrain.iloc[:,-1]

def remove(text):
  return "".join([c for c in text if c not in string.punctuation])

commentsTrain = commentsTrain.apply(lambda x: remove(x))

tokenizer = RegexpTokenizer(r'\w+')
commentsTrain = commentsTrain.apply(lambda x: tokenizer.tokenize(x.lower()))

def stop(text):
  return [w for w in text if w not in stopwords.words('english')]

commentsTrain = commentsTrain.apply(lambda x: stop(x))

In [0]:
lemmatizer = WordNetLemmatizer()

def lem(text):
  return [lemmatizer.lemmatize(i) for i in text]

commentsTrain = commentsTrain.apply(lambda x: lem(x))

In [0]:
stem = PorterStemmer()
def stemmer(text):
  return " ".join([stem.stem(i) for i in text])

In [0]:
commentsTrain = commentsTrain.apply(lambda x: stemmer(x))

In [0]:
GX_train = commentsTrain[commentsTrain.index < np.percentile(commentsTrain.index, 80)].sort_index()
Gy_train = subredditsTrain[subredditsTrain.index < np.percentile(subredditsTrain.index, 80)].sort_index().values
GX_test = commentsTrain[commentsTrain.index > np.percentile(commentsTrain.index, 80)].sort_index()
Gy_test = subredditsTrain[subredditsTrain.index > np.percentile(subredditsTrain.index, 80)].sort_index().values

In [0]:
tfidf = TfidfVectorizer(stop_words='english')
GX_train_idf = tfidf.fit_transform(GX_train)
GX_test_idf = tfidf.transform(GX_test)

In [0]:
clf = MultinomialNB(alpha=0.31005)

In [96]:
clf.fit(GX_train_idf,Gy_train)
# print(GX_train_idf.shape)
# print(Gy_train.shape)

MultinomialNB(alpha=0.31005, class_prior=None, fit_prior=True)

In [97]:
clf.score(GX_test_idf, Gy_test)

0.5640714285714286

In [98]:
redditDataTest = pd.read_csv("/content/reddit_test.csv") 
commentsTest = redditDataTest.iloc[:,1]
subredditsTest = redditDataTest.iloc[:,-1]

commentsTest = commentsTest.apply(lambda x: remove(x))

commentsTest = commentsTest.apply(lambda x: tokenizer.tokenize(x.lower()))

commentsTest = commentsTest.apply(lambda x: stop(x))


commentsTest = commentsTest.apply(lambda x: lem(x))

commentsTest = commentsTest.apply(lambda x: stemmer(x))


NotFittedError: ignored

In [0]:
pred = clf.predict(commentsTest)
pd.DataFrame(pred, columns=['Category']).to_csv("testResultsPreprocess.csv")